# 第9章: 複雑なプロンプトをゼロから作成する

- [レッスン](#レッスン)
- [演習](#演習)
- [例題プレイグラウンド](#例題プレイグラウンド)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
import json

# utilsパッケージからhintsモジュールをインポート
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": prefill}
            ],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

最後の章まで来られたことおめでとうございます！今からすべてをまとめて、**ユニークで複雑なプロンプトを作成する**方法を学びます。

以下では、**複雑なプロンプトに推奨するガイド付きの構造**を使用します。この章の後半では、いくつかの業界固有のプロンプトを紹介し、それらのプロンプトがどのように同様に構造化されているかを説明します。

**注意:** **すべてのプロンプトが以下の複雑な構造のすべての要素を必要とするわけではありません**。要素を含めたり除外したりして、それがClaudeの応答にどのように影響するかを試してみることをお勧めします。通常は、**多くのプロンプト要素を使用してプロンプトを機能させてから、後からプロンプトを洗練させて簡素化するのが最良の方法です**。

### 例 - キャリアコーチチャットボット

以下の構造は、複数のプロンプトエンジニアリング要素を組み合わせたもので、複雑なプロンプトの良い出発点となります。**一部の要素では順序が重要**ですが、そうでないものもあります。ベストプラクティスが順序の重要性を示している場合は注記しますが、一般的に、**この順序に従えば、素晴らしいプロンプトの良いスタートとなります**。

以下の例では、Claudeが特定のタスクを持つ状況的な役割を担うコントロールされたロールプレイのためのプロンプトを構築します。目標は、Claudeにフレンドリーなキャリアコーチとして行動するようプロンプトすることです。

以下のセルを読んでから実行して、さまざまなプロンプト要素を1つの完全なプロンプトにまとめてください。

In [ ]:
######################################## 入力変数 ########################################

# 最初の入力変数 - 会話履歴（これはAPIコールの前の`user`および`assistant`メッセージとして追加することもできます）
HISTORY = """Customer: 社会学専攻の可能な職業を2つ挙げてください。

Joe: 社会学専攻の方に向けて、以下の2つの潜在的なキャリアを挙げることができます：

- ソーシャルワーカー - 社会学は人間行動や社会システムを理解するための強固な基盤を提供します。追加の訓練や資格取得により、社会学の学位は個人やグループを支援するソーシャルワーカー、ケースマネージャー、カウンセラー、コミュニティオーガナイザーなどの役割に就くことができます。

- 人事スペシャリスト - 社会学から得られるグループダイナミクスや組織行動の理解は、人事の分野でのキャリアに適用できます。卒業生は採用、従業員関係、研修開発、多様性とインクルージョン、その他の人事機能において役割を見つけることができます。社会構造や制度に焦点を当てることで、公共政策、非営利団体管理、教育などの関連するキャリアも支援します。"""

# 2つ目の入力変数 - ユーザーの質問
QUESTION = "この2つのキャリアのうち、学士号以上の学位が必要なのはどちらですか？"



######################################## プロンプト要素 ########################################

##### プロンプト要素1: `user`の役割
# メッセージ配列の最初に必ず`user`の役割を置くようにしてください。
# 上記で定義されたget_completion()関数は、自動的にこれを行います。

##### プロンプト要素2: タスクの文脈
# Claudeに、引き受けるべき役割や、プロンプトで実行したい目標や全体的なタスクについての文脈を与えます。
# プロンプトの本文の早い段階で文脈を提供するのが最良です。
TASK_CONTEXT = "あなたはAdAstra Careers社が作成したJoeという名前のAIキャリアコーチとして行動します。あなたの目標はユーザーにキャリアアドバイスを提供することです。あなたはAdAstraサイトにいるユーザーに返信することになります。ユーザーは、あなたがJoeとしての性格で返答しないと混乱するでしょう。"

##### プロンプト要素3: トーンの文脈
# インタラクションに重要な場合は、Claudeが使用すべきトーンを伝えます。
# この要素はタスクによっては必要ない場合があります。
TONE_CONTEXT = "フレンドリーなカスタマーサービスのトーンを維持してください。"

##### プロンプト要素4: 詳細なタスクの説明とルール
# Claudeに実行してほしい具体的なタスク、およびClaudeが従うべきルールを詳しく説明します。
# これは、Claudeに答えがない場合や分からない場合の「逃げ道」を与える場所でもあります。
# この説明とルールを友人に見せて、論理的に配置されているか、曖昧な言葉が明確に定義されているかを確認するのが理想的です。
TASK_DESCRIPTION = """以下はインタラクションに関する重要なルールです：
- 常にAdAstra CareersのAIであるJoeとしての性格を保ってください
- 回答方法がわからない場合は、「申し訳ありません。それをよく理解できませんでした。質問を言い換えていただけますか？」と言ってください
- 誰かが関係のないことを質問した場合は、「申し訳ありません。私はJoeで、キャリアアドバイスを提供しています。今日、キャリアに関する質問はありますか？」と言ってください"""

##### プロンプト要素5: 例
# Claudeに模倣できる理想的な回答の例を少なくとも1つ提供します。これを<example></example>XMLタグで囲みます。複数の例を提供しても構いません。
# 複数の例を提供する場合は、それが何の例であるかをClaudeに説明し、各例を独自のXMLタグセットで囲みます。
# 例は、Claudeに望ましい行動をとらせるための知識作業において、おそらく最も効果的なツールです。
# 一般的なエッジケースの例をClaudeに提供するようにしてください。プロンプトでスクラッチパッドを使用する場合、スクラッチパッドがどのように見えるべきかの例を提供するのが効果的です。
# 一般的に、例が多いほど良いです。
EXAMPLES = """以下は標準的なインタラクションでの返答方法の例です：
<example>
Customer: こんにちは、あなたはどのように作られ、何をするのですか？
Joe: こんにちは！私の名前はJoeで、AdAstra Careersによってキャリアアドバイスを提供するために作られました。今日は何をお手伝いできますか？
</example>"""

##### プロンプト要素6: 処理する入力データ
# プロンプト内でClaudeが処理する必要のあるデータがある場合、ここで関連するXMLタグ内に含めます。
# 複数のデータを含めても構いませんが、それぞれを独自のXMLタグセットで囲むようにしてください。
# この要素はタスクによっては必要ない場合があります。順序も柔軟です。
INPUT_DATA = f"""以下は質問に先立つ会話履歴（ユーザーとあなたの間）です。履歴がない場合は空になる可能性があります：
<history>
{HISTORY}
</history>

以下はユーザーの質問です：
<question>
{QUESTION}
</question>"""

##### プロンプト要素7: 即時のタスク説明または要求 #####
# Claudeに、プロンプトのタスクを実行するために即座に行うべきことを「リマインド」または正確に伝えます。
# ここでは、ユーザーの質問などの追加の変数を入れることもできます。
# Claudeに即時のタスクを繰り返し伝えても一般的に害はありません。長いプロンプトの終わりに近い方が良いです。
# これは、最初に置くよりも良い結果をもたらします。
# また、ユーザーのクエリをプロンプトの下部に近い位置に置くのも一般的に良い習慣です。
IMMEDIATE_TASK = "ユーザーの質問にどのように答えますか？"

##### プロンプト要素8: 予知（ステップバイステップで考える）
# 複数のステップを含むタスクの場合、答えを出す前にステップバイステップで考えるようClaudeに伝えるのが良いです
# 場合によっては、「答えを出す前に...」と言って、Claudeが最初にこれを行うようにする必要があるかもしれません。
# すべてのプロンプトで必要というわけではありませんが、含める場合は長いプロンプトの終わりに近く、最終的な即時タスクの要求や説明の直後に行うのが最良です。
PRECOGNITION = "回答する前に、まず答えについて考えてください。"

##### プロンプト要素9: 出力フォーマット
# Claudeの応答を特定の方法でフォーマットしたい場合は、そのフォーマットを明確にClaudeに伝えます。
# この要素はタスクによっては必要ない場合があります。
# 含める場合は、プロンプトの最初よりも終わりに近い方が良いです。
OUTPUT_FORMATTING = "回答を<response></response>タグで囲んでください。"

##### プロンプト要素10: Claudeの応答の事前入力（ある場合）
# Claudeの回答を事前に入力された単語で始めて、Claudeの行動や応答を導くためのスペースです。
# Claudeの応答を事前に入力したい場合は、APIコールの`assistant`の役割にこれを入れる必要があります。
# この要素はタスクによっては必要ない場合があります。
PREFILL = "[Joe] <response>"



######################################## 要素の結合 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 完全なプロンプトを出力
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーのターン")
print(PROMPT)
print("\nアシスタントのターン")
print(PREFILL)

では、プロンプトを実行しましょう！以下のセルを実行してClaudeの出力を確認してください。

In [ ]:
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

### 例 - 法律サービス

**法律専門家のプロンプトは非常に複雑**になる可能性があります。その理由は以下の通りです：
- 長文書を解析する必要がある
- 複雑なトピックを扱う
- 出力を非常に特定の方法でフォーマットする
- 多段階の分析プロセスに従う

複雑なプロンプトテンプレートを使用して、特定の法律ユースケースのプロンプトを構造化する方法を見てみましょう。以下では、法律文書の情報を使用して法律問題に関する質問に答えるようClaudeに求める法律ユースケースの例プロンプトを詳しく説明しています。

プロンプト構造が柔軟であることを示すために、**いくつかの要素の順序を変更しました**！

**プロンプトエンジニアリングは科学的な試行錯誤に関するものです**。要素を混ぜ合わせたり、移動したり（順序が重要でない要素）して、あなたとあなたのニーズに最適なものを見つけることをお勧めします。

In [ ]:
######################################## 入力変数 ########################################

# 最初の入力変数 - 法律文書
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
動物健康産業は、過去1年間に多くの特許および商標訴訟に巻き込まれました。1994年、Barclay Slocumは、犬の前十字靭帯断裂の治療に使用される脛骨高平部水平化骨切り術の手順と、その手順で使用される装置の特許を取得しました。2006年、Slocum Enterprisesは、New Generation Devicesが製造するUnity Cruciate PlateがSlocum TPLO plateの特許を侵害していると主張し、特許侵害訴訟を提起しました。しかし、裁判所は特許侵害の問題について判断を下すことはありませんでした。訴訟が提起された州で販売されたプレートの数が少なかったこと、およびSlocum Enterprisesが維持するウェブサイトで提供された情報に基づいて、管轄権がないと判断したためです。2006年に行われた他の特許戦では、猫の爪切除術におけるレーザー技術の使用、ペット識別チップ、豚のワクチン、ペットの「脱毛」ツールが争点となりました。
</search_result>
<search_result id=2>
カナダでは、ブリティッシュコロンビア獣医師会が非獣医師を訴え、彼が馬の歯のフックを切断または除去し、電動および手動工具で馬の歯を浮かせ、これらの活動に関して報酬を得て助言や診断を提供し、これらの活動に関して治療を提供する資格があり、willing であると自称していたと主張しました。裁判所は、獣医専門職法を制定した立法府の意図が公衆と動物の保護であったと判断し、さらに独占的な法令が公衆を保護する目的を果たすと判断しました。さらに、裁判所は、歯科は本質的に歯と歯茎の健康に関連し、動物の美容やその他のタイプのケアとは区別され、したがって獣医医療の実践の定義に含まれると結論付けました。非獣医師は、獣医師が手順を監督することなくサービスを提供することを禁じられました。
</search_result>
<search_result id=3>
2005年に米国のメキシコ湾岸を襲ったハリケーン・カトリーナの余波は、自然災害時の動物の扱い方に変化をもたらしました。2006年、ハワイ、ルイジアナ、ニューハンプシャーの各州は、ペットのための避難所の提供や、介助動物をその主人と一緒に避難させることなど、災害時の動物のケアに関する問題に対処する法律を制定しました。さらに、2006年に連邦議会は「ペット避難・輸送基準法」を可決し、大統領が署名しました。この法律は、州および地方の緊急事態対策当局に対し、災害時にペットおよび介助動物をどのように収容するかについての情報を避難計画に含めることを要求しています。カリフォルニア州は、緊急事態対応準備に関与する緊急サービス局、農務省、その他の機関に対し、災害や重大な緊急事態の際の介助動物、家畜、馬科動物、ペットのニーズに対応する計画を策定することを要求する法律を可決しました。
</search_result>
</search_results>"""

# 2つ目の入力変数 - ユーザーの質問
QUESTION = "ハリケーンの際にペットをどうするかについての法律はありますか？"



######################################## プロンプト要素 ########################################

##### プロンプト要素1: `user`の役割
# メッセージ配列の最初に必ず`user`の役割を置くようにしてください。
# 上記で定義されたget_completion()関数は、自動的にこれを行います。

##### プロンプト要素2: タスクの文脈
# Claudeに、引き受けるべき役割や、プロンプトで実行したい目標や全体的なタスクについての文脈を与えます。
# プロンプトの本文の早い段階で文脈を提供するのが最良です。
TASK_CONTEXT = "あなたは専門の弁護士です。"

##### プロンプト要素3: トーンの文脈
# インタラクションに重要な場合は、Claudeが使用すべきトーンを伝えます。
# この要素はタスクによっては必要ない場合があります。
TONE_CONTEXT = ""

##### プロンプト要素4: 処理する入力データ
# プロンプト内でClaudeが処理する必要のあるデータがある場合、ここで関連するXMLタグ内に含めます。
# 複数のデータを含めても構いませんが、それぞれを独自のXMLタグセットで囲むようにしてください。
# この要素はタスクによっては必要ない場合があります。順序も柔軟です。
INPUT_DATA = f"""以下は収集された研究です。これを使用してユーザーからの法的質問に答えてください。
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### プロンプト要素5: 例
# Claudeに模倣できる理想的な回答の例を少なくとも1つ提供します。これを<example></example>XMLタグで囲みます。複数の例を提供しても構いません。
# 複数の例を提供する場合は、それが何の例であるかをClaudeに説明し、各例を独自のXMLタグセットで囲みます。
# 例は、Claudeに望ましい行動をとらせるための知識作業において、おそらく最も効果的なツールです。
# 一般的なエッジケースの例をClaudeに提供するようにしてください。プロンプトでスクラッチパッドを使用する場合、スクラッチパッドがどのように見えるべきかの例を提供するのが効果的です。
# 一般的に、例が多いほど良いです。
EXAMPLES = """回答で法律研究を引用する際は、検索インデックスIDを含む角括弧を使用し、それに続けてピリオドを付けてください。これらは引用している文の最後に配置してください。適切な引用形式の例：

<examples>
<example>
このような犯罪の時効は10年後に切れます。[3]。
</example>
<example>
ただし、両当事者が特別に権利放棄した場合、この保護は適用されません。[5]。
</example>
</examples>"""

##### プロンプト要素6: 詳細なタスクの説明とルール
# Claudeに実行してほしい具体的なタスク、およびClaudeが従うべきルールを詳しく説明します。
# これは、Claudeに答えがない場合や分からない場合の「逃げ道」を与える場所でもあります。
# この説明とルールを友人に見せて、論理的に配置されているか、曖昧な言葉が明確に定義されているかを確認するのが理想的です。
TASK_DESCRIPTION = """以下の質問に対して、明確で簡潔な回答を書いてください：

<question>
{QUESTION}
</question>

回答は数段落以内にしてください。可能であれば、ユーザーの質問に直接答える1文で結論付けてください。ただし、提供された研究資料に十分な情報がない場合は、「申し訳ありませんが、この質問に答えるための十分な情報を持ち合わせていません」と書いても構いません。"""

##### プロンプト要素7: 即時のタスク説明または要求 #####
# Claudeに、プロンプトのタスクを実行するために即座に行うべきことを「リマインド」または正確に伝えます。
# ここでは、ユーザーの質問などの追加の変数を入れることもできます。
# Claudeに即時のタスクを繰り返し伝えても一般的に害はありません。長いプロンプトの終わりに近い方が良いです。
# これは、最初に置くよりも良い結果をもたらします。
# また、ユーザーのクエリをプロンプトの下部に近い位置に置くのも一般的に良い習慣です。
IMMEDIATE_TASK = ""

##### プロンプト要素8: 予知（ステップバイステップで考える）
# 複数のステップを含むタスクの場合、答えを出す前にステップバイステップで考えるようClaudeに伝えるのが良いです
# 場合によっては、「答えを出す前に...」と言って、Claudeが最初にこれを行うようにする必要があるかもしれません。
# すべてのプロンプトで必要というわけではありませんが、含める場合は長いプロンプトの終わりに近く、最終的な即時タスクの要求や説明の直後に行うのが最良です。
PRECOGNITION = "回答する前に、研究から最も関連性の高い引用を<relevant_quotes>タグ内に抽出してください。"

##### プロンプト要素9: 出力フォーマット
# Claudeの応答を特定の方法でフォーマットしたい場合は、そのフォーマットを明確にClaudeに伝えます。
# この要素はタスクによっては必要ない場合があります。
# 含める場合は、プロンプトの最初よりも終わりに近い方が良いです。
OUTPUT_FORMATTING = "2段落の回答を<answer>タグで囲んでください。"

##### プロンプト要素10: Claudeの応答の事前入力（ある場合）
# Claudeの回答を事前に入力された単語で始めて、Claudeの行動や応答を導くためのスペースです。
# Claudeの応答を事前に入力したい場合は、APIコールの`assistant`の役割にこれを入れる必要があります。
# この要素はタスクによっては必要ない場合があります。
PREFILL = "<relevant_quotes>"



######################################## 要素の結合 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 完全なプロンプトを出力
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーのターン")
print(PROMPT)
print("\nアシスタントのターン")
print(PREFILL)

では、プロンプトを実行しましょう！以下のセルを実行してClaudeの出力を確認してください。

In [ ]:
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

上記のコンテンツを変更せずにレッスンプロンプトを試してみたい場合は、レッスンノートブックの最後まで下にスクロールして[**例題プレイグラウンド**](#例題プレイグラウンド)をご覧ください。

---

## 演習
- [演習 9.1 - 金融サービスチャットボット](#exercise-91---financial-services-chatbot)
- [演習 9.2 - コードボット](#exercise-92---codebot)

### 演習9.1 - 金融サービスチャットボット
金融業界におけるプロンプトも、法律のプロンプトと同様の理由で非常に複雑になる可能性があります。ここでは、Claudeを使用して**税務情報を分析し、質問に答える**金融ユースケースの演習を行います。法務サービスの例と同様に、解決策のプロンプトでは異なる流れの方が理にかなっているため、いくつかの要素の順序を変更しています（ただし、他の構造も機能します）。

変数の内容（この場合は`{QUESTION}`と`{TAX_CODE}`）を読み、Claudeが扱うことが期待される内容を理解することをお勧めします。プロンプトのどこかで`{QUESTION}`と`{TAX_CODE}`を直接参照するようにしてください（他の例と同様にf-string構文を使用）。これにより、実際の変数内容を置き換えることができます。

プロンプト要素のフィールドに、説明と、これまでに見た複雑なプロンプトの例に合致する内容を記入してください。すべてのプロンプト要素を記入したら、セルを実行して連結されたプロンプトとClaudeの応答を確認してください。

プロンプトエンジニアリングは、特に大規模で複雑なプロンプトの場合、純粋に公式化されることはほとんどないことを覚えておいてください！テストケースを開発し、**各状況に最適なものを見つけるために、さまざまなプロンプトやプロンプト構造を試してみる**ことが重要です。プロンプト要素の順序を変更する場合は、`COMBINE ELEMENTS`セクションでの連結の順序も変更することを忘れないでください。

In [ ]:
######################################## 入力変数 ########################################

# 最初の入力変数 - ユーザーの質問
QUESTION = "83b選択をするのにどのくらいの期間がありますか？"

# 2番目の入力変数 - Claudeがユーザーの質問に答えるために使用する税法文書
TAX_CODE = """
(a)一般規則
サービスの提供に関連して、そのサービスを受ける者以外の者に財産が譲渡される場合、以下の超過額—
(1)その財産の公正市場価値（決して失効しない条件以外の制限を考慮せずに決定される）が、受益者の権利が譲渡可能となるか、または没収の実質的なリスクがなくなる時点のいずれか早い方の時点での価値、から
(2)その財産に対して支払われた金額（もしあれば）を引いた額、
は、そのサービスを提供した者の総所得に含まれるものとし、その財産に対する受益者の権利が譲渡可能となるか、または没収の実質的なリスクがなくなる最初の課税年度のいずれか適用される年度に含まれるものとする。前文は、その者がその財産に対する権利が譲渡可能となる前、または没収の実質的なリスクがなくなる前に、独立当事者間取引でその財産を売却またはその他の方法で処分した場合には適用されないものとする。
(b)譲渡年度に総所得に含める選択
(1)一般
財産が譲渡されるサービスを提供する者は、その財産が譲渡される課税年度の総所得に以下の超過額を含めることを選択することができる—
(A)譲渡時のその財産の公正市場価値（決して失効しない条件以外の制限を考慮せずに決定される）、から
(B)その財産に対して支払われた金額（もしあれば）を引いた額。
この選択がなされた場合、(a)項はその財産の譲渡に関して適用されず、その財産がその後没収された場合、その没収に関して控除は認められないものとする。
(2)選択
財産の譲渡に関する(1)項に基づく選択は、財務長官が規定する方法で行われ、その譲渡の日から30日以内に行われなければならない。この選択は、財務長官の同意がない限り取り消すことはできない。

(c)特別規則
本節の目的のために—
(1)没収の実質的なリスク
ある者の財産に対する権利は、その財産の完全な享受に対するその者の権利が、個人による将来の実質的なサービスの提供を条件とする場合、没収の実質的なリスクの対象となる。

(2)財産の譲渡可能性
ある者の財産に対する権利は、その財産に対する譲受人の権利が没収の実質的なリスクの対象とならない場合にのみ譲渡可能である。

(3)1934年証券取引法第16(b)条に基づく訴訟を引き起こす可能性のある売却
財産の利益での売却が1934年証券取引法第16(b)条に基づく訴訟の対象となる可能性がある限り、その財産に対するその者の権利は—
(A)没収の実質的なリスクの対象となり、
(B)譲渡不可能である。
(4)サービスの提供に関連して譲渡された財産に対する個人の基礎を決定する目的のために、第72条(w)項の規則に類似した規則が適用されるものとする。
(d)決して失効しない特定の制限
(1)評価
その条件上決して失効せず、譲受人が公式で決定された価格でのみその財産を売却することを許可する制限の対象となる財産の場合、そのように決定された価格は、財務長官によって反対が立証されない限り、その財産の公正市場価値とみなされるものとし、その価値に関する立証責任は財務長官にあるものとする。

(2)取消
その条件上決して失効しない制限の対象となる財産の場合で、その制限が取り消される場合、納税者が以下を立証しない限り—
(A)その取消が報酬的でなかったこと、および
(B)その取消が報酬的に扱われた場合に控除を認められる者（もしいれば）が、財務長官が規則で規定する方法で示されるように、その取引を報酬的でないものとして扱うこと、
取消時の財産の公正市場価値（制限を考慮せずに計算）から以下の合計を引いた超過額—
(C)取消直前のその財産の公正市場価値（制限を考慮して計算）、および
(D)取消に対して支払われた金額（もしあれば）、
は、その取消が発生する課税年度の報酬として扱われるものとする。
(e)節の適用性
本節は以下には適用されない—
(1)第421条が適用される取引、
(2)第401条(a)に記述される信託への、または信託からの譲渡、または第404条(a)(2)の要件を満たす年金制度に基づく譲渡、
(3)容易に確定可能な公正市場価値を持たないオプションの譲渡、
(4)付与日に容易に確定可能な公正市場価値を持つオプションの行使に従った財産の譲渡、または
(5)第79条が適用されるグループ定期生命保険。
(f)保有期間
(a)項が適用される財産について納税者が保有した期間を決定する際、その者の財産に対する権利が譲渡可能となるか、または没収の実質的なリスクの対象とならなくなる最初の時点のいずれか早い方から始まる期間のみが含まれるものとする。

(g)特定の交換
(a)項が適用される財産が、その交換で与えられた財産が従っていた制限および条件と実質的に類似した制限および条件の対象となる財産と交換され、かつ第354条、355条、356条、または1036条（または第1036条に関連する第1031条の部分）がその交換に適用された場合、またはその交換が転換特権の行使に従って行われた場合—
(1)その交換は(a)項の目的のために無視されるものとし、
(2)受け取った財産は(a)項が適用される財産として扱われるものとする。
(h)雇用主による控除
本節が適用される財産の譲渡または(d)項に記述される制限の取消の場合、その財産が譲渡されたサービスを提供した者のために、第162条に基づく控除が認められるものとし、その金額は(a)項、(b)項、または(d)(2)項に基づいてそのサービスを提供した者の総所得に含まれる金額と等しいものとする。この控除は、そのサービスを提供した者の総所得にその金額が含まれる課税年度が終了する、またはその課税年度内の、その者の課税年度に認められるものとする。

(i)適格株式付与
(1)一般
本副題の目的のために—
(A)含める時期
適格株式が、本項に基づいてその株式に関して選択を行う適格従業員に譲渡される場合、(a)項は、その株式に関して(a)項に基づいて決定された金額を(B)号に基づいて決定された課税年度の従業員の所得に含めることによって適用されるものとし、(a)項に記述される課税年度の代わりとする。

(B)決定される課税年度
本号に基づいて決定される課税年度は、以下のいずれか最も早い日を含む従業員の課税年度である—
(i)その適格株式が譲渡可能となる最初の日（この句の目的のためにのみ、雇用主に譲渡可能となることを含む）、
(ii)従業員が除外従業員となる日、
(iii)適格株式を発行した法人の株式が確立された証券市場で容易に取引可能となる最初の日（財務長官によって決定されるが、本項以外の本編の規定の目的で財務長官によって確立された証券市場として認識されない限り、いかなる市場も含まない）、
(iv)従業員のその株式に対する権利が譲渡可能となるか、または没収の実質的なリスクの対象とならなくなる最初の日から5年後の日、またはいずれか早い方、または
(v)従業員がその株式に関する本項に基づく選択を取り消す日（財務長官が規定する時期および方法で）。
(2)適格株式
(A)一般
本項の目的のために、「適格株式」という用語は、任意の適格従業員に関して、その従業員の雇用主である法人の株式を意味し、以下の場合に該当する—
(i)その株式が—
(I)オプションの行使に関連して、または
(II)制限付株式ユニットの決済において、受け取られ、
(ii)そのオプションまたは制限付株式ユニットが法人によって付与された—
(I)従業員としてのサービス提供に関連して、かつ
(II)その法人が適格法人であった暦年中に。
(B)制限
「適格株式」という用語には、従業員がその株式に対する権利が最初に譲渡可能となるか、または没収の実質的なリスクの対象とならなくなる時点で、その株式を法人に売却する、またはその他の方法で法人から株式の代わりに現金を受け取ることができる場合、その株式は含まれないものとする。

(C)適格法人
(A)(ii)(II)号の目的のために—
(i)一般
「適格法人」という用語は、任意の暦年に関して、以下の場合に該当する法人を意味する—
(I)その法人（またはその法人の前身）の株式が、いかなる先行する暦年中も、(1)(B)(iii)項に基づいて決定される確立された証券市場で容易に取引可能でない、かつ
(II)その法人が、その暦年中に、米国（または米国の属領）でその法人にサービスを提供するすべての従業員の80%以上に、同じ権利および特権を持って適格株式を受け取るストックオプションを付与する、または制限付株式ユニットを付与する、という書面による計画を持っている。
(ii)同じ権利および特権
(i)(II)句の目的のために—
(I)(II)および(III)項に規定される場合を除き、株式に関する権利および特権の決定は、第423条(b)(5)項に基づく方法と同様の方法で行われるものとする、
(II)すべての従業員が利用可能な株式数が同じでない場合でも、各従業員が利用可能な株式数が些細な量以上である限り、従業員は適格株式を受け取る同じ権利および特権を持っていないとは扱われないものとする、および
(III)オプションの行使に関する権利および特権は、制限付株式ユニットの決済に関する権利および特権と同じとは扱われないものとする。
(iii)従業員
(i)(II)句の目的のために、「従業員」という用語には、第4980E条(d)(4)項に記述される従業員または除外従業員は含まれないものとする。

(iv)2018年以前の暦年に関する特別規則
2018年1月1日より前に始まる暦年の場合、(i)(II)句は、適格株式に関する権利および特権が同じであるかどうかを考慮せずに適用されるものとする。

(3)適格従業員；除外従業員
本項の目的のために—
(A)一般
「適格従業員」という用語は、以下の個人を意味する—
(i)除外従業員ではない、および
(ii)本項に基づいて行われる選択において、適格株式に関する第24章に基づく法人の源泉徴収要件が満たされることを確実にするために必要であると財務長官が決定する要件を満たすことに同意する。
(B)除外従業員
「除外従業員」という用語は、任意の法人に関して、以下の個人を意味する—
(i)暦年中のいかなる時点でも1パーセント所有者（第416条(i)(1)(B)(ii)項の意味の範囲内）である、または先行する10暦年中のいかなる時点でもそのような1パーセント所有者であった者、
(ii)以下の者である、または過去にそうであった者—
(I)その法人の最高経営責任者またはそのような立場で行動する個人、または
(II)その法人の最高財務責任者またはそのような立場で行動する個人、
(iii)第318条(a)(1)項に記述される関係を(ii)号の(I)項または(II)項に記述される個人と持つ者、または
(iv)その法人の課税年度における4人の最高報酬役員の1人である、または先行する10課税年度のいずれかにおける4人の最高報酬役員の1人であった者で、1934年証券取引法に基づく報酬に関する株主開示規則に基づいて（そのような規則がその法人に適用されたかのように）各そのような課税年度に関して決定される。
(4)選択
(A)選択を行う時期
適格株式に関する選択は、従業員のその株式に対する権利が譲渡可能となるか、または没収の実質的なリスクの対象とならなくなる最初の日から30日以内に本項に基づいて行われなければならず、(b)項に基づく選択が行われる方法と同様の方法で行われるものとする。

(B)制限
以下の場合、適格株式に関して本節に基づく選択を行うことはできない—
(i)適格従業員がその適格株式に関して(b)項に基づく選択を行っている場合、
(ii)その選択が行われる前のいかなる時点でも、適格株式を発行した法人の株式が(1)(B)(iii)項に基づいて決定される確立された証券市場で容易に取引可能である場合、または
(iii)その法人が、従業員のその株式に対する権利が最初に譲渡可能となるか、または没収の実質的なリスクの対象とならなくなる最初の日を含む暦年の前の暦年に、その発行済み株式を購入した場合。ただし、以下の場合を除く—
(I)そのように購入された株式の総ドル金額の25%以上が繰延株式である場合、および
(II)繰延株式を購入する個人の決定が合理的な基準で行われる場合。
(C)株式の買戻しに関する制限に関連する定義および特別規則
(i)繰延株式
本項の目的のために、「繰延株式」という用語は、本項に基づく選択が有効である株式を意味する。

(ii)個人が長い繰延期間を持つ繰延株式を保有している場合、その個人からの株式の購入は考慮されない
法人が個人から購入した株式は、その個人が（購入直後に）その購入された株式に関する選択よりも長い期間、本項に基づく選択が有効であった繰延株式を保有している場合、(B)(iii)号の目的のために繰延株式として扱われないものとする。

(iii)すべての発行済み繰延株式の購入
(B)(iii)号の(I)項および(II)項の要件は、購入された株式にその法人のすべての発行済み繰延株式が含まれる場合、満たされたものとして扱われるものとする。

(iv)報告
任意の暦年の開始時点で繰延株式を発行済みとし、その暦年中にその発行済み株式を購入する法人は、その暦年が終了する、またはその暦年を含む課税年度の税務申告書に、その暦年中にそのように購入されたその発行済み株式の総ドル金額および本項を管理する目的で財務長官が要求するその他の情報を含めなければならない。

(5)支配グループ
本項の目的のために、第414条(b)項に基づいて単一の雇用主として扱われるすべての者は、1つの法人として扱われるものとする。

(6)通知要件
適格株式を適格従業員に譲渡する法人は、その株式に起因する金額が（本項がなければ）その従業員の総所得に最初に含まれる時点（またはその前の合理的な期間）に—
(A)その株式が適格株式であることをその従業員に証明し、
(B)その従業員に以下を通知するものとする—
(i)従業員が本項に基づいてその株式に関する所得の繰延を選択する資格がある可能性があること、および
(ii)従業員がそのような選択を行う場合—
(I)繰延期間の終了時に認識される所得の金額は、従業員のその株式に対する権利が最初に譲渡可能となるか、または没収の実質的なリスクの対象とならなくなる時点での株式の価値に基づくものとし、繰延期間中に株式の価値が下落したとしてもそれにかかわらず、
(II)繰延期間の終了時に認識される所得の金額は、第3402条(t)項に基づいて決定される税率で第3401条(i)項に基づく源泉徴収の対象となること、および
(III)そのような源泉徴収に関する従業員の責任（(3)(A)(ii)項に基づいて財務長官によって決定される）。
(7)制限付株式ユニット
本節（本項を除く）は、(b)項に基づく選択を含め、制限付株式ユニットには適用されないものとする。
"""



######################################## プロンプト要素 ########################################

##### プロンプト要素1: `user`ロール
# メッセージAPIコールが常にメッセージ配列の`user`ロールから始まることを確認してください。
# 上記で定義されたget_completion()関数は、これを自動的に行います。

##### プロンプト要素2: タスクの文脈
# Claudeに、引き受けるべき役割や、プロンプトで取り組んでほしい目標や全体的なタスクについての文脈を与えてください。
# 文脈はプロンプトの本文の早い段階に置くのが最適です。
TASK_CONTEXT = ""

##### プロンプト要素3: トーンの文脈
# インタラクションにとって重要な場合、Claudeが使用すべきトーンを伝えてください。
# このタスクによっては、この要素が必要ない場合もあります。
TONE_CONTEXT = ""

##### プロンプト要素4: 処理する入力データ
# プロンプト内でClaudeが処理する必要のあるデータがある場合、ここに関連するXMLタグ内に含めてください。
# 複数のデータを含めても構いませんが、それぞれを独自のXMLタグで囲むようにしてください。
# タスクによっては、この要素が必要ない場合もあります。順序も柔軟です。
INPUT_DATA = ""

##### プロンプト要素5: 例
# Claudeに模倣できる理想的な応答の例を少なくとも1つ提供してください。これを<example></example>XMLタグで囲んでください。複数の例を提供しても構いません。
# 複数の例を提供する場合、それが何の例であるかClaudeに文脈を与え、各例を独自のXMLタグで囲んでください。
# 例は、Claudeに望む行動をさせるための最も効果的なツールの1つです。
# 一般的なエッジケースの例をClaudeに提供するようにしてください。プロンプトがスクラッチパッドを使用する場合、スクラッチパッドがどのように見えるべきかの例を提供するのが効果的です。
# 一般的に、例が多いほど良いでしょう。
EXAMPLES = ""

##### プロンプト要素6: 詳細なタスクの説明とルール
# Claudeに行ってほしい具体的なタスク、およびClaudeが従う必要があるルールを詳しく説明してください。
# ここでは、Claudeが答えを持っていない、または知らない場合の「逃げ道」を与えることもできます。
# この説明とルールを友人に見せて、論理的に配置されているか、曖昧な言葉が明確に定義されているかを確認するのが理想的です。
TASK_DESCRIPTION = ""

##### プロンプト要素7: 即時のタスクの説明または要求 #####
# プロンプトのタスクを遂行するために、Claudeに期待されることを「リマインド」または正確に伝えてください。
# ここは、ユーザーの質問などの追加の変数を入れる場所でもあります。
# 一般的に、長いプロンプトの終わりに向かってClaudeに即時のタスクを繰り返すのは害にはなりません。これは、
# 最初に置くよりも良い結果をもたらすでしょう。
# また、ユーザーのクエリをプロンプトの下部に近い位置に置くのも一般的に良い習慣です。
IMMEDIATE_TASK = ""

##### プロンプト要素8: 予知（段階的思考）
# 複数のステップを持つタスクの場合、答えを出す前に段階的に考えるようClaudeに伝えるのが良いでしょう
# 時には、「答えを出す前に...」と言う必要があるかもしれません。これは、Claudeが最初にこれを行うことを確実にするためです。
# すべてのプロンプトで必要というわけではありませんが、含める場合は、長いプロンプトの終わりに向かって、最終的な即時のタスクの要求や説明の直後に行うのが最適です。
PRECOGNITION = ""

##### プロンプト要素9: 出力フォーマット
# Claudeの応答を特定の方法でフォーマットしたい場合は、そのフォーマットを明確にClaudeに伝えてください。
# このタスクによっては、この要素が必要ない場合もあります。
# 含める場合は、プロンプトの始めよりも終わりに向かって置く方が良いでしょう。
OUTPUT_FORMATTING = ""

##### プロンプト要素10: Claudeの応答の事前入力（ある場合）
# Claudeの行動や応答を導くために、Claudeの答えを事前に入力した言葉で始めるためのスペースです。
# Claudeの応答を事前に入力したい場合は、APIコールの`assistant`ロールにこれを入れる必要があります。
# このタスクによっては、この要素が必要ない場合もあります。
PREFILL = ""



######################################## 要素の結合 ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# 完全なプロンプトを表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン")
print(PROMPT)
print("\nアシスタントターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ If you want to see a possible solution, run the cell below!

In [ ]:
print(hints.exercise_9_1_solution)

### Exercise 9.2 - Codebot
In this exercise, we will write up a prompt for a **coding assistance and teaching bot that reads code and offers guiding corrections when appropriate**. Fill in the prompt element fields with content that match the description and the examples you've seen in the preceding examples of complex prompts. Once you have filled out all the prompt elements that you want to fill out, run the cell to see the concatenated prompt as well as Claude's response.

We suggest you read through the variable content (in this case, `{CODE}`) to understand what content Claude is expected to work with. Be sure to reference `{CODE}` directly in your prompt somewhere (using f-string syntax like in the other examples) so that the actual variable content can be substituted in.

In [ ]:
######################################## INPUT VARIABLES ########################################

# Input variable - the code that Claude needs to read and assist the user with correcting
CODE = """
# Function to print multiplicative inverses
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = ""

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = ""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = ""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = ""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = ""

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = ""

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = ""



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

❓ If you want to see a possible solution, run the cell below!

In [ ]:
print(hints.exercise_9_2_solution)

---

## Congratulations & Next Steps!

If you made it through all the exercises, **you are now in the top 0.1% of LLM whisperers**. One of the elite!

The techniques you've learned, from thinking step by step to assigning roles to using examples to general all-around clear writing, can be **merged, remixed, and adapted in countless ways**.

Prompt engineering is a very new discipline, so keep an open mind. You could be the one to discover the next great prompting trick.

If you want to see **more examples of good prompts** for inspiration:					
- Learn from examples of production-ready prompts from our [cookbook](https://anthropic.com/cookbook)
- Read through our [prompting guide](https://docs.anthropic.com/claude/docs/prompt-engineering)
- Check out our [prompt library](https://anthropic.com/prompts) for inspiration
- Try our experimental [metaprompt](https://docs.anthropic.com/claude/docs/helper-metaprompt-experimental) to get Claude to write prompt templates for you!
- Ask questions in our [discord server](https://anthropic.com/discord)
- Learn about the [Anthropic API parameters](https://docs.anthropic.com/claude/reference/complete_post) like temperature and `max_tokens`
- If you're feeling academic, read some [papers](https://www.promptingguide.ai/papers) on prompt engineering
- Practice building prompts to get Claude to do something you're interested in

If you want to learn about some truly advanced prompting techniques beyond the scope of this tutorial, click through to the appendix! But first, run the cell below.

In [ ]:
# Prompt
PROMPT = "Write an ode to a fabulous student who has just completed a course on prompt engineering, in the form of a sonnet."

# Print Claude's response
print(get_completion(PROMPT))

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the conversation history (this can also be added as preceding `user` and `assistant` messages in the API call)
HISTORY = """Customer: Give me two possible careers for sociology majors.

Joe: Here are two potential careers for sociology majors:

- Social worker - Sociology provides a strong foundation for understanding human behavior and social systems. With additional training or certification, a sociology degree can qualify graduates for roles as social workers, case managers, counselors, and community organizers helping individuals and groups.

- Human resources specialist - An understanding of group dynamics and organizational behavior from sociology is applicable to careers in human resources. Graduates may find roles in recruiting, employee relations, training and development, diversity and inclusion, and other HR functions. The focus on social structures and institutions also supports related careers in public policy, nonprofit management, and education."""

# Second input variable - the user's question
QUESTION = "Which of the two careers requires more than a Bachelor's degree?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You will be acting as an AI career coach named Joe created by the company AdAstra Careers. Your goal is to give career advice to users. You will be replying to users who are on the AdAstra site and who will be confused if you don't respond in the character of Joe."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = "You should maintain a friendly customer service tone."

##### Prompt element 4: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Here are some important rules for the interaction:
- Always stay in character, as Joe, an AI from AdAstra Careers
- If you are unsure how to respond, say \"Sorry, I didn't understand that. Could you rephrase your question?\"
- If someone asks something irrelevant, say, \"Sorry, I am Joe and I give career advice. Do you have a career question today I can help you with?\""""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """Here is an example of how to respond in a standard interaction:
<example>
Customer: Hi, how were you created and what do you do?
Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
</example>"""

##### Prompt element 6: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
<history>
{HISTORY}
</history>

Here is the user's question:
<question>
{QUESTION}
</question>"""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = "How do you respond to the user's question?"

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Think about your answer first before you respond."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your response in <response></response> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "[Joe] <response>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
######################################## INPUT VARIABLES ########################################

# First input variable - the legal document
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
The animal health industry became caught up in a number of patent and trademark lawsuits during the past year. In 1994, Barclay Slocum obtained patents for the tibial plateau leveling osteotomy procedure, which is used in the treatment of dogs with cranial cruciate ligament rupture, and for the devices used in the procedure. During 2006, Slocum Enterprises filed a patent infringement suit against New Generation Devices, arguing that the Unity Cruciate Plate manufactured by New Generation infringed on the patent for the Slocum TPLO plate. However, the court never reached a decision on the issue of patent infringement, ruling that it did not have jurisdiction on the basis of the small number of plates sold in the state in which the case was filed and the information provided on a Web site maintained by Slocum Enterprises. Other patent battles waged during 2006 concerned the use of laser technology for onychectomy in cats, pet identification chips, pig vaccines, and pet “deshedding” tools.
</search_result>
<search_result id=2>
In Canada, the British Columbia Veterinary Medical Association brought suit against a nonveterinarian, claiming that he engaged in cutting or otherwise removing hooks from horses' teeth and floating horses' teeth with power and manual tools, provided advice and diagnoses in return for a fee, and held himself out as being qualified and willing to provide treatment with respect to these activities. The court held that the intention of the legislature in passing the Veterinary Profession Act was the protection of the public and animals and further held that monopolistic statutes serve the purpose of protecting the public. In addition, the court concluded that dentistry, at its core, relates to the health of the teeth and gums; is distinct from cosmetic and other types of care of animals; and, therefore, falls under the definition of the practice of veterinary medicine. The nonveterinarian was enjoined from providing services without a veterinarian supervising the procedures.
</search_result>
<search_result id=3>
The aftermath of Hurricane Katrina, which hit the Gulf Coast of the United States during 2005, spurred changes to the way animals are treated during natural disasters. In 2006, Hawaii, Louisiana, and New Hampshire all enacted laws that address issues regarding the care of animals during disasters, such as providing shelters for pets and allowing service animals to be kept with the people they serve. In addition, Congress passed, and the President signed, the Pet Evacuation and Transportation Standards Act during 2006, which requires state and local emergency preparedness authorities to include in their evacuation plans information on how they will accommodate household pets and service animals in case of a disaster. California passed a law that will require its Office of Emergency Services, Department of Agriculture, and other agencies involved with disaster response preparation to develop a plan for the needs of service animals, livestock, equids, and household pets in the event of a disaster or major emergency.
</search_result>
</search_results>"""

# Second input variable - the user's question
QUESTION = "Are there any laws about what to do with pets during a hurricane?"



######################################## PROMPT ELEMENTS ########################################

##### Prompt element 1: `user` role
# Make sure that your Messages API call always starts with a `user` role in the messages array.
# The get_completion() function as defined above will automatically do this for you.

##### Prompt element 2: Task context
# Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
# It's best to put context early in the body of the prompt.
TASK_CONTEXT = "You are an expert lawyer."

##### Prompt element 3: Tone context
# If important to the interaction, tell Claude what tone it should use.
# This element may not be necessary depending on the task.
TONE_CONTEXT = ""

##### Prompt element 4: Input data to process
# If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
# Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
# This element may not be necessary depending on task. Ordering is also flexible.
INPUT_DATA = f"""Here is some research that's been compiled. Use it to answer a legal question from the user.
<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""

##### Prompt element 5: Examples
# Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
# If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
# Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
# Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
# Generally more examples = better.
EXAMPLES = """When citing the legal research in your answer, please use brackets containing the search index ID, followed by a period. Put these at the end of the sentence that's doing the citing. Examples of proper citation format:

<examples>
<example>
The statute of limitations expires after 10 years for crimes like this. [3].
</example>
<example>
However, the protection does not apply when it has been specifically waived by both parties. [5].
</example>
</examples>"""

##### Prompt element 6: Detailed task description and rules
# Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
# This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
# It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
TASK_DESCRIPTION = """Write a clear, concise answer to this question:

<question>
{QUESTION}
</question>

It should be no more than a couple of paragraphs. If possible, it should conclude with a single sentence directly answering the user's question. However, if there is not sufficient information in the compiled research to produce such an answer, you may demur and write "Sorry, I do not have sufficient information at hand to answer this question."."""

##### Prompt element 7: Immediate task description or request #####
# "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
# This is also where you would put in additional variables like the user's question.
# It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
# This will yield better results than putting this at the beginning.
# It is also generally good practice to put the user's query close to the bottom of the prompt.
IMMEDIATE_TASK = ""

##### Prompt element 8: Precognition (thinking step by step)
# For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
# Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
# Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
PRECOGNITION = "Before you answer, pull out the most relevant quotes from the research in <relevant_quotes> tags."

##### Prompt element 9: Output formatting
# If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
# This element may not be necessary depending on the task.
# If you include it, putting it toward the end of the prompt is better than at the beginning.
OUTPUT_FORMATTING = "Put your two-paragraph response in <answer> tags."

##### Prompt element 10: Prefilling Claude's response (if any)
# A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
# If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
# This element may not be necessary depending on the task.
PREFILL = "<relevant_quotes>"



######################################## COMBINE ELEMENTS ########################################

PROMPT = ""

if TASK_CONTEXT:
    PROMPT += f"""{TASK_CONTEXT}"""

if TONE_CONTEXT:
    PROMPT += f"""\n\n{TONE_CONTEXT}"""

if INPUT_DATA:
    PROMPT += f"""\n\n{INPUT_DATA}"""

if EXAMPLES:
    PROMPT += f"""\n\n{EXAMPLES}"""

if TASK_DESCRIPTION:
    PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

if IMMEDIATE_TASK:
    PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

if PRECOGNITION:
    PROMPT += f"""\n\n{PRECOGNITION}"""

if OUTPUT_FORMATTING:
    PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

# Print full prompt
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))